In [2]:
import pandas as pd
import py4cytoscape as p4c
import time
import requests

In [3]:
p4c.__version__

'1.6.0'

In [4]:
p4c.cytoscape_ping()

You are connected to Cytoscape!


'You are connected to Cytoscape!'

In [48]:
def create3Dnetwork(networkLayers, stylexml):
    layerTable = pd.read_table(networkLayers, header=None, sep="\t")
    networkSUID = layerTable.apply(importLayer, axis=1)
    layerTable = pd.concat([layerTable, networkSUID], axis=1)
    nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)
    layeredNodes = getLayeredNodes(nodetables)
    edgetables = layerTable.apply(getEdgeTableWithLayerinfo, axis=1)
    layeredEdges = pd.concat(edgetables.tolist())
    layeredNodes.to_csv("layeredNodes.csv")
    layeredEdges.to_csv("layeredEdges.csv")
    p4c.commands_post('cy3d set renderer')
    suID = p4c.create_network_from_data_frames(layeredNodes.astype(str), layeredEdges.astype(str))
    setTransomicStyle(stylexml, suID)
    return suID

def importLayer(row: pd.Series) -> int:
    r = requests.get("https://rest.kegg.jp/get/" + row.array[1] + "/kgml")
    kgml = row.array[1] + ".xml"
    f = open(kgml, "w")
    f.write(r.text)
    f.close()
    print("Importing " + kgml)
    res = p4c.import_network_from_file(kgml)
    #time.sleep(3)
    xy = p4c.get_table_columns(table='node', columns=['KEGG_NODE_X', 'KEGG_NODE_Y'])
    xy.rename(columns={"KEGG_NODE_X": "x_location", "KEGG_NODE_Y": "y_location"}, inplace=True)
    p4c.load_table_data(xy, table_key_column="SUID")
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_X_LOCATION", "x_location", "p"))
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_Y_LOCATION", "y_location", "p"))
    p4c.fit_content()
    if row.array[3]:
        print("Creating a node at each midpoint of the edge...")
        createMidnodes(res['networks'][0])
    return res['networks'][0]

def getNodeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    nt = p4c.get_table_columns(table="node", network=row.array[4])
    if 'x_location' not in nt.columns:
        xy = p4c.get_table_columns(table="node", network=row.array[4], columns=['x_location', 'y_location'])
        nt = pd.concat([nt, xy], axis=1)
    nt['z_location'] = row.array[2]
    nt['LAYER_INDEX'] = row.array[0]
    return nt

def getLayeredNodes(nodetables: pd.DataFrame) -> pd.DataFrame:
    nodetable3d = pd.concat(nodetables.array)
    return nodetable3d.rename(columns={"SUID": "id"})

def getEdgeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    et = p4c.get_table_columns(table="edge", network=row.array[4])
    #return et
    print("Getting edge info. This function is kinda slow...")
    ei = p4c.get_edge_info(et["SUID"].tolist(), network=row.array[4])
    print("Finished getting edge info.")
    et['source'] = list(map(lambda x: x['source'], ei))
    et['target'] = list(map(lambda x: x['target'], ei))
    et['LAYER_INDEX'] = row.array[0]
    return et

def createMidnodes(networkSUID):
    et = p4c.get_table_columns(table='edge', network=networkSUID)
    print("getting node positions...")
    xyloc = p4c.get_table_columns(table='node', network=networkSUID, columns=['SUID', 'x_location', 'y_location'])
    print("getting edge information...")
    einfo = p4c.get_edge_info(et['SUID'].tolist(), network=networkSUID)
    einfodf = pd.DataFrame.from_dict(einfo)
    midxydf = pd.DataFrame.from_dict([getMidLoc(x, xyloc) for x in einfo])
    midNodeInfo = einfodf.join(midxydf)    
    midNodeInfo.rename(columns={"SUID": "orig_edge_SUID"}, inplace=True)
    print("creating midpoint nodes...")
    midNodes = p4c.add_cy_nodes(et['SUID'].apply(str).to_list())
    midNodeInfo['orig_edge_SUID'] = midNodeInfo['orig_edge_SUID'].apply(str)
    p4c.load_table_data(midNodeInfo, data_key_column="orig_edge_SUID", table="node", table_key_column="name")
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_X_LOCATION", "x_location", "p"))
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_Y_LOCATION", "y_location", "p"))
    print("deleting edges...")
    p4c.invert_edge_selection()
    p4c.delete_selected_edges()
    print("start collecting node IDs to connect...")
    midNodesName2suid =  pd.DataFrame.from_dict(midNodes)
    midNodeInfo = pd.merge(midNodeInfo, midNodesName2suid, left_on='orig_edge_SUID', right_on='name')
    src2mid = midNodeInfo[['source', 'SUID']]
    src2mid.rename(columns={"SUID": "target"}, inplace=True)
    mid2tgt = midNodeInfo[['SUID', 'target']]
    mid2tgt.rename(columns={"SUID": "source"}, inplace=True)
    res = p4c.cyrest_post("networks/" + str(networkSUID) + "/edges", body=src2mid.to_dict('records'))
    res = p4c.cyrest_post("networks/" + str(networkSUID) + "/edges", body=mid2tgt.to_dict('records'))

def getMidLoc(elem, xyloc):
    sourcexy = xyloc[xyloc['SUID'] == elem['source']]
    targetxy = xyloc[xyloc['SUID'] == elem['target']]
    midx_location = (int(sourcexy['x_location'].values[0]) + int(targetxy['x_location'].values[0])) / 2
    midy_location = (int(sourcexy['y_location'].values[0]) + int(targetxy['y_location'].values[0])) / 2
    return {'x_location': midx_location, 'y_location': midy_location}

def setTransomicStyle(xml, suid) -> None:
    #xml is must be a local file path
    stylename = p4c.import_visual_styles(xml)
    res = p4c.set_visual_style(stylename[0], network = suid)
    print(stylename[0] + " " + res['message'])

In [49]:
suid = create3Dnetwork("https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv", "yugi2014.xml")

Importing rno04910.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Importing rno01100.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Creating a node at each midpoint of the edge...
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316
getting node positions...
getting edge information...
creating midpoint nodes...


...

deleting edges...
start collecting node IDs to connect...
Importing rno01100.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74
Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 2632
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316
Getting edge info. This function is kinda slow...
Finished getting edge info.


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Applying default style...
Applying preferred layout
yugi2014 Visual Style applied.


In [10]:
suid

,0,1,2,3,0
0,kinase-layer,rno04910,1500,False,124
1,enzyme-layer,rno01100,900,True,1162
2,metabolite-layer,rno01100,1,False,48267


In [11]:
nt = p4c.get_table_columns(table="node", network=124)

Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74


In [12]:
nt

,SUID,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST
258,258,path:rno04910:17,path:rno04910:17,False,642,693,46,17,Mapk1...,Mapk1...
261,261,path:rno04910:18,path:rno04910:18,False,956,230,46,17,Fasn,Fasn
264,264,path:rno04910:19,path:rno04910:19,False,875,222,46,17,Prkag1...,Prkag1...
267,267,path:rno04910:20,path:rno04910:20,False,981,408,8,8,C00369,Starch
270,270,path:rno04910:21,path:rno04910:21,False,824,510,8,8,C00575,cAMP
...,...,...,...,...,...,...,...,...,...,...
243,243,path:rno04910:12,path:rno04910:12,False,135,551,46,17,Ptprf,Ptprf
246,246,path:rno04910:13,path:rno04910:13,False,334,295,46,17,Ptpn1,Ptpn1
249,249,path:rno04910:14,path:rno04910:14,False,334,268,46,17,Socs1...,Socs1...
252,252,path:rno04910:15,path:rno04910:15,False,714,705,46,17,Elk1,Elk1


In [13]:
nt = p4c.get_table_columns(table="node", network=1162)

Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637


In [14]:
nt

,SUID,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST
24576,24576,rn:R03245,rn:R03245,False,None,None,None,None,None,None
8193,8193,path:rno01100:3405,path:rno01100:3405,False,1018,2010,14,14,C01724,Lanosterol
24578,24578,rn:R03286 rn:R03288,rn:R03286 rn:R03288,False,None,None,None,None,None,None
8196,8196,path:rno01100:3406,path:rno01100:3406,False,1076,2010,14,14,C11455,"4,4-Dimethyl-5alpha-cholesta-8,14,24-trien-3be..."
24580,24580,rn:R04738,rn:R04738,False,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
24570,24570,rn:R05795,rn:R05795,False,None,None,None,None,None,None
8187,8187,path:rno01100:3403,path:rno01100:3403,False,894,2010,14,14,C00751,Squalene
24572,24572,rn:R01677 rn:R02147,rn:R01677 rn:R02147,False,None,None,None,None,None,None
8190,8190,path:rno01100:3404,path:rno01100:3404,False,952,2010,14,14,C01054,"2,3-Oxidosqualene"


In [15]:
a = p4c.get_table_column_types(table="node", network=1162)

In [16]:
a

{'SUID': 'Long',
 'shared name': 'String',
 'name': 'String',
 'selected': 'Boolean',
 'KEGG_NODE_X': 'String',
 'KEGG_NODE_Y': 'String',
 'KEGG_NODE_WIDTH': 'String',
 'KEGG_NODE_HEIGHT': 'String',
 'KEGG_NODE_LABEL': 'String',
 'KEGG_NODE_LABEL_LIST_FIRST': 'String',
 'KEGG_NODE_LABEL_LIST': 'List',
 'KEGG_ID': 'List',
 'KEGG_NODE_LABEL_COLOR': 'String',
 'KEGG_NODE_FILL_COLOR': 'String',
 'KEGG_NODE_REACTIONID': 'String',
 'KEGG_NODE_TYPE': 'String',
 'KEGG_NODE_SHAPE': 'String',
 'KEGG_LINK': 'String',
 'row.names': 'Integer',
 'x_location': 'String',
 'y_location': 'String',
 'source': 'Integer',
 'target': 'Integer',
 'orig_edge_SUID': 'String',
 'shared interaction': 'String',
 'interaction': 'String',
 'KEGG_REACTION_TYPE': 'String',
 'KEGG_REACTION_GENE': 'String'}

In [21]:
list(a)

['SUID',
 'shared name',
 'name',
 'selected',
 'KEGG_NODE_X',
 'KEGG_NODE_Y',
 'KEGG_NODE_WIDTH',
 'KEGG_NODE_HEIGHT',
 'KEGG_NODE_LABEL',
 'KEGG_NODE_LABEL_LIST_FIRST',
 'KEGG_NODE_LABEL_LIST',
 'KEGG_ID',
 'KEGG_NODE_LABEL_COLOR',
 'KEGG_NODE_FILL_COLOR',
 'KEGG_NODE_REACTIONID',
 'KEGG_NODE_TYPE',
 'KEGG_NODE_SHAPE',
 'KEGG_LINK',
 'row.names',
 'x_location',
 'y_location',
 'source',
 'target',
 'orig_edge_SUID',
 'shared interaction',
 'interaction',
 'KEGG_REACTION_TYPE',
 'KEGG_REACTION_GENE']

In [33]:
nt = p4c.get_table_columns(table="node", network=1162, columns=list(a))

Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637


In [35]:
list(a)[8:13]

['KEGG_NODE_LABEL',
 'KEGG_NODE_LABEL_LIST_FIRST',
 'KEGG_NODE_LABEL_LIST',
 'KEGG_ID',
 'KEGG_NODE_LABEL_COLOR']

In [36]:
nt = p4c.get_table_columns(table="node", network=1162, columns=list(a)[8:13])

Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637


In [46]:
nt = p4c.get_table_columns(table="node", network=1162, columns=['x_location', 'y_location'])

In [47]:
nt

,x_location,y_location
24576,1297.5,1956.0
8193,1018,2010
24578,3604.0,928.0
8196,1076,2010
24580,2036.0,1665.5
...,...,...
24570,1857.5,1220.0
8187,894,2010
24572,4082.0,428.0
8190,952,2010


In [44]:
nt = p4c.get_table_columns(table="node", network=1162, columns=list(a)[:10])

In [45]:
nt

,SUID,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST
24576,24576,rn:R03245,rn:R03245,False,None,None,None,None,None,None
8193,8193,path:rno01100:3405,path:rno01100:3405,False,1018,2010,14,14,C01724,Lanosterol
24578,24578,rn:R03286 rn:R03288,rn:R03286 rn:R03288,False,None,None,None,None,None,None
8196,8196,path:rno01100:3406,path:rno01100:3406,False,1076,2010,14,14,C11455,"4,4-Dimethyl-5alpha-cholesta-8,14,24-trien-3be..."
24580,24580,rn:R04738,rn:R04738,False,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
24570,24570,rn:R05795,rn:R05795,False,None,None,None,None,None,None
8187,8187,path:rno01100:3403,path:rno01100:3403,False,894,2010,14,14,C00751,Squalene
24572,24572,rn:R01677 rn:R02147,rn:R01677 rn:R02147,False,None,None,None,None,None,None
8190,8190,path:rno01100:3404,path:rno01100:3404,False,952,2010,14,14,C01054,"2,3-Oxidosqualene"


In [11]:
def getMidLoc(elem, xyloc):
    sourcexy = xyloc[xyloc['SUID'] == elem['source']]
    targetxy = xyloc[xyloc['SUID'] == elem['target']]
    midx_location = (int(sourcexy['x_location'].values[0]) + int(targetxy['x_location'].values[0])) / 2
    midy_location = (int(sourcexy['y_location'].values[0]) + int(targetxy['y_location'].values[0])) / 2
    #elem['x_location'] = midx_location
    #elem['y_location'] = midy_location
    #return elem
    return {'x_location': midx_location, 'y_location': midy_location}
    #return pd.Series({'x_location': midx_location, 'y_location': midy_location})

In [14]:
layerTable = pd.concat([layerTable, networkSUID], axis=1)

In [15]:
layerTable

,0,1,2,3,0
0,kinase-layer,rno04910,1500,False,124
1,enzyme-layer,rno01100,900,True,1162
2,metabolite-layer,rno01100,1,False,48267


In [66]:
def getNodeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    nt = p4c.get_table_columns(table="node", network=row.array[4])
    if 'x_location' not in nt.columns:
        xy = p4c.get_table_columns(table="node", network=row.array[4], columns=['x_location', 'y_location'])
        nt = pd.concat([nt, xy], axis=1)
    nt['z_location'] = row.array[2]
    nt['LAYER_INDEX'] = row.array[0]
    return nt

In [67]:
nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)

Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74
Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637


In [69]:
nodetables[0]

,SUID,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST,x_location,y_location,z_location,LAYER_INDEX
258,258,path:rno04910:17,path:rno04910:17,False,642,693,46,17,Mapk1...,Mapk1...,642,693,1500,kinase-layer
261,261,path:rno04910:18,path:rno04910:18,False,956,230,46,17,Fasn,Fasn,956,230,1500,kinase-layer
264,264,path:rno04910:19,path:rno04910:19,False,875,222,46,17,Prkag1...,Prkag1...,875,222,1500,kinase-layer
267,267,path:rno04910:20,path:rno04910:20,False,981,408,8,8,C00369,Starch,981,408,1500,kinase-layer
270,270,path:rno04910:21,path:rno04910:21,False,824,510,8,8,C00575,cAMP,824,510,1500,kinase-layer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,243,path:rno04910:12,path:rno04910:12,False,135,551,46,17,Ptprf,Ptprf,135,551,1500,kinase-layer
246,246,path:rno04910:13,path:rno04910:13,False,334,295,46,17,Ptpn1,Ptpn1,334,295,1500,kinase-layer
249,249,path:rno04910:14,path:rno04910:14,False,334,268,46,17,Socs1...,Socs1...,334,268,1500,kinase-layer
252,252,path:rno04910:15,path:rno04910:15,False,714,705,46,17,Elk1,Elk1,714,705,1500,kinase-layer


In [41]:
table_col_list = list(table_col_info.keys())

In [46]:
import py4cytoscape.py4cytoscape_utils

In [48]:
py4cytoscape.py4cytoscape_utils.DEFAULT_BASE_URL

'http://127.0.0.1:1234/v1'

In [50]:
res_names = p4c.commands.cyrest_get(f'networks/124/tables/defaultnode/columns/SUID', base_url='http://127.0.0.1:1234/v1')

In [57]:
res_col = p4c.commands.cyrest_get(f'networks/124/tables/defaultnode/columns/x_location')

In [58]:
res_col

{'name': 'x_location',
 'values': ['642',
  '956',
  '875',
  '981',
  '824',
  '64',
  '981',
  '956',
  '770',
  '956',
  '870',
  '870',
  '884',
  '956',
  '616',
  '770',
  '956',
  '770',
  '511',
  '354',
  '426',
  '770',
  '570',
  '210',
  '582',
  '135',
  '616',
  '282',
  '770',
  '616',
  '498',
  '956',
  '956',
  '511',
  '770',
  '956',
  '582',
  '844',
  '282',
  '616',
  '616',
  '334',
  '482',
  '770',
  '770',
  '382',
  '956',
  '382',
  '616',
  '210',
  '925',
  '516',
  '282',
  '234',
  '421',
  '135',
  '160',
  '1080',
  '1082',
  '1082',
  '1005',
  '365',
  '349',
  '616',
  '616',
  '334',
  '334',
  '334',
  '334',
  '135',
  '334',
  '334',
  '714',
  '714']}

In [83]:
def create3Dnetwork(networkLayers, stylexml):
    layerTable = pd.read_table(networkLayers, header=None, sep="\t")
    networkSUID = layerTable.apply(importLayer, axis=1)
    layerTable = pd.concat([layerTable, networkSUID], axis=1)
    nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)
    layeredNodes = getLayeredNodes(nodetables)
    edgetables = layerTable.apply(getEdgeTableWithLayerinfo, axis=1)
    layeredEdges = pd.concat(edgetables.tolist())
    # layeredNodes.to_csv("layeredNodes.csv")
    # layeredEdges.to_csv("layeredEdges.csv")
    p4c.commands_post('cy3d set renderer')
    suID = p4c.create_network_from_data_frames(layeredNodes.astype(str), layeredEdges.astype(str))
    setTransomicStyle(stylexml, suID)
    return suID

In [36]:
midNodeInfo['orig_edge_SUID'] = midNodeInfo['orig_edge_SUID'].apply(str)

In [27]:
xyloc = p4c.get_table_columns(table='node', network=1162, columns=['SUID', 'x_location', 'y_location'])

In [39]:
(xyloc['SUID'] == 7638).value_counts()

False    3320
True        1
Name: SUID, dtype: int64

In [40]:
sourcexy = xyloc[xyloc['SUID'] == 7638]

In [88]:
def getLayeredNodes(nodetables: pd.DataFrame) -> pd.DataFrame:
    nodetable3d = pd.concat(nodetables.array)
    return nodetable3d.rename(columns={"SUID": "id"})

In [89]:
def getEdgeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    et = p4c.get_table_columns(table="edge", network=row.array[4])
    #return et
    print("Getting edge info. This function is kinda slow...")
    ei = p4c.get_edge_info(et["SUID"].tolist(), network=row.array[4])
    print("Finished getting edge info.")
    et['source'] = list(map(lambda x: x['source'], ei))
    et['target'] = list(map(lambda x: x['target'], ei))
    et['LAYER_INDEX'] = row.array[0]
    return et